In [7]:
import glob
import os
import sys
import numpy as np
import pandas as pd

import scanpy as sc
import pandas as pd

from scipy.io import mmread
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import gzip
import re
import mygene

import tqdm.notebook as tq
import matplotlib.pyplot as plt

In [2]:
path = "/n/data1/hms/cellbio/sander/judy/resource_paper/"

# FrangiehIzar2021

In [ ]:
### Done!

In [3]:
index = "FrangiehIzar2021"

In [9]:
adata = sc.read_h5ad(index+"/"+index+"/"+index+"_protein_preprocessed.h5")
adata.var.index = [x.split(" ")[0] for x in adata.var.Protein]
adata.var.index = adata.var.index.rename('protein')
adata.obs = adata.obs.rename({"condition":"perturbation_2"},axis=1)
adata.obs = adata.obs.rename({"sgRNAs":"guide_id"},axis=1)

In [10]:
def find_pert(sgRNAs):
    if sgRNAs == "nan":
        return "control"
    else:
        sgRNAs = sgRNAs.split(",")
        if sum(["_SITE" in x for x in sgRNAs]) == len(sgRNAs):
            return "control"
        else:
            sgRNAs = "_".join([x.split("_")[0] for x in sgRNAs if "_SITE" not in x])
            return sgRNAs

In [12]:
adata.obs["guide_id"] = [";".join(x.split(",")) for x in adata.obs["guide_id"]]
adata.obs["perturbation"] = [find_pert(x) for x in adata.obs.guide_id]
adata.obs['tissue_type']='cell_line_co-culture'
adata.obs['cancer'] = True
adata.obs['disease'] = 'melanoma'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'melanocytes'
adata.obs['organism'] = 'human'
adata.obs['perturbation_type_2'] = 'IFN-γ stimulation'
adata.obs.index = adata.obs.NAME
adata.obs.index = adata.obs.index.rename('cell_name')
adata.obs = adata.obs.drop(['NAME'],axis=1)

In [13]:
abinfo = pd.read_excel(index+"/"+index+"/41588_2021_779_MOESM3_ESM.xlsx", 
                       sheet_name="Supplementary Table 3", skiprows=2, header=0)

In [14]:
abinfo.index = ['CD117', 'CD119', 'CD140a', 'CD140b', 'CD172a', 'CD184', 'CD202b', 'CD274', 
                'CD279', 'CD29', 'CD309', 'CD44', 'CD47', 'CD49f', 'CD58', 'CD59', 'CD61', 
                'CD9', 'HLA_A', 'HLA_E', 'Mouse_IgG1', 'Mouse_IgG2a', 'Mouse_IgG2b', 'Rat_IgG2a']

In [15]:
var = adata.var.merge(abinfo, left_index=True, right_index=True)

In [16]:
var = var.drop(["Protein",'# '], axis=1)

In [17]:
var.columns=["Target","Clone","TotalSeq A#","Barcode"]

In [18]:
var['Isotype_control'] = ["Mouse_IgG1","Mouse_IgG1","Mouse_IgG1","Mouse_IgG1","Mouse_IgG2a","Mouse_IgG2a",
                          "Mouse_IgG1","Mouse_IgG2b","Mouse_IgG1","Mouse_IgG1","Mouse_IgG1","Mouse_IgG1",
                          "Rat_IgG2a","Mouse_IgG1","Mouse_IgG2a","Mouse_IgG1","Mouse_IgG2a","nan",
                          "nan","nan","nan","Mouse_IgG1","Mouse_IgG1","Mouse_IgG1"]

In [19]:
adata.var = var
adata.var.index = adata.var.index.rename('protein')

In [20]:
adata.write(index+"/"+index+"/"+index+"_protein.h5")

... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation_type_2' as categorical
... storing 'Isotype_control' as categorical


In [35]:
!/home/cs308/dropbox_uploader.sh upload {index+"/"+index+"/"+index+"_protein.h5"} {index+"/"+index+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/FrangiehIzar2021/FrangiehIzar2021/FrangiehIzar2021_protein.h5" to "/FrangiehIzar2021/FrangiehIzar2021/FrangiehIzar2021_protein.h5"... DONE


In [23]:
adata = sc.read_h5ad(index+"/"+index+"/"+index+"_protein.h5")
obs = adata.obs

In [21]:
adata = sc.read_h5ad(index+"/"+index+"/"+index+"_RNA_full_preprocessed.h5")
adata.var.index = adata.var.gene

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [11]:
mg = mygene.MyGeneInfo()
query = mg.querymany(list(set(adata.var["gene"])) , scopes='symbol', fields='ensembl.gene', species='human', returnall=True, verbose=False)
gene_symbols = [x['query'] for x in query['out']]
ensembl_id = [x['ensembl'] if 'ensembl' in x.keys() else np.nan for x in query['out']]
ensembl_id = [x['gene'] if type(x)==dict else x for x in ensembl_id]
ensembl_id = [x[0]['gene'] if type(x)==list else x for x in ensembl_id]

In [27]:
df = pd.DataFrame({"gene":gene_symbols,"ensembl_id":ensembl_id})
df.index = gene_symbols
df = df.drop_duplicates(keep='first')

for dupidx in np.where(df.index.duplicated())[0]:
    ensemblids = df["ensembl_id"].iloc[dupidx-1:dupidx+1]
    if sum(~pd.isna(ensemblids)) == 1:
        ensemblid = ensemblids[~pd.isna(ensemblids)]
        df["ensembl_id"].iloc[dupidx-1:dupidx+1] = ensemblid
df = df.drop_duplicates(subset="gene", keep='first')

In [28]:
adata.var["ensembl_id"] = [df.loc[x,"ensembl_id"] for x in adata.var.index]
adata.var.loc[pd.isna(adata.var['ensembl_id']),'ensembl_id'] = adata.var.gene[pd.isna(adata.var['ensembl_id'])]
adata.var.index = adata.var.index.rename('gene_symbol')
adata.var = adata.var.drop(['gene'],axis=1)

In [ ]:
adata.obs = adata.obs.rename({"condition":"perturbation_2"},axis=1)

In [32]:
adata.obs

,NAME,library_preparation_protocol,condition,MOI,sgRNA,UMI_count,sgRNAs,umap_x,umap_y
0,CELL_1,10X 3' v3 sequencing,Control,1,HLA-B_2,10832.0,HLA-B_2,-3.937523,5.441272
1,CELL_2,10X 3' v3 sequencing,Control,2,nan,10731.0,"NGFR_3,SERPINF1_3",-5.480882,6.598254
2,CELL_3,10X 3' v3 sequencing,Control,1,HLA-B_2,28821.0,HLA-B_2,-1.044962,9.840532
3,CELL_4,10X 3' v3 sequencing,Control,2,nan,15322.0,"NMRK1_3,S100A6_3",-2.588797,6.588357
4,CELL_5,10X 3' v3 sequencing,Control,0,nan,10314.0,nan,-0.795585,2.256979
...,...,...,...,...,...,...,...,...,...
218326,CELL_218327,10X 3' v3 sequencing,Co-culture,4,nan,12722.0,"CTSO_3,PIK3IP1_3,VDAC2_2,WNT7A_1",10.117681,-1.233309
218327,CELL_218328,10X 3' v3 sequencing,Co-culture,2,nan,11643.0,"JAK2_3,SLC7A5P1_3",5.348833,0.995789
218328,CELL_218329,10X 3' v3 sequencing,Co-culture,3,nan,12497.0,"S100A6_2,SAT1_2,ONE_NON-GENE_SITE_177",6.285855,1.838057
218329,CELL_218330,10X 3' v3 sequencing,Co-culture,1,CDKN2B_3,17941.0,CDKN2B_3,7.731683,1.007936


In [33]:
adata.obs["perturbation"] = [find_pert(x) for x in adata.obs.sgRNAs]
adata.obs['tissue_type']='cell_line_co-culture'
adata.obs['cancer'] = True
adata.obs['disease'] = 'melanoma'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'melanocytes'
adata.obs['organism'] = 'human'
adata.obs['perturbation_type_2'] = 'IFN-γ stimulation'
adata.obs.index = adata.obs.NAME
adata.obs.index = adata.obs.index.rename('cell_name')
adata.obs = adata.obs.drop(['NAME'],axis=1)

In [34]:
adata.write(index+"/"+index+"/"+index+"_RNA_normalized.h5")

... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation_type_2' as categorical


In [4]:
adata = sc.read(index+"/"+index+"/"+index+"_RNA_normalized.h5")

In [5]:
tmp = adata.X.expm1()
tmpraw = tmp.T.multiply(np.array(adata.obs.UMI_count/1000000))
tmpcsr = tmpraw.tocsr().T
tmpcsrfinal = tmpcsr.rint()

In [6]:
tmpcsrfinal

<218331x23712 sparse matrix of type '<class 'numpy.float64'>'
	with 740736244 stored elements in Compressed Sparse Column format>

In [7]:
adatanew = sc.AnnData(tmpcsrfinal, obs=adata.obs, var=adata.var)
adatanew.write(index+"/"+index+"/"+index+"_RNA.h5")

In [28]:
adata = sc.read_h5ad(index+"/"+index+"/"+index+"_RNA.h5")
adata.obs = obs
adata.write(index+"/"+index+"/"+index+"_RNA.h5")

In [36]:
!/home/cs308/dropbox_uploader.sh upload {index+"/"+index+"/"+index+"_RNA.h5"} {index+"/"+index+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/FrangiehIzar2021/FrangiehIzar2021/FrangiehIzar2021_RNA.h5" to "/FrangiehIzar2021/FrangiehIzar2021/FrangiehIzar2021_RNA.h5" by 114 chunks .................................................................................................................. DONE


# GasperiniShendure2019

In [ ]:
# Done!

In [3]:
index = "GasperiniShendure2019"

In [109]:
experiments = ["lowMOI","highMOI","atscale"]

In [110]:
def find_pert(gene):
    if tmp=="nan":
        pert="control"
    else:
        perts = gene.split("_")
        scrambleidx = np.where(np.array(perts)=="scrambled")[0]
        if len(scrambleidx) > 0:
            idxkeep = np.delete(np.arange(len(perts)),np.array(list(scrambleidx) + list(scrambleidx+1)))
            perts = [perts[x] for x in idxkeep]
        perts = [x for x in perts if x!="TSS"]
        pert = "_".join(perts)
        if pert == "":
            pert = "control"
    return pert

In [111]:
for experiment in experiments:
#     X = csr_matrix(mmread(path+index+"/"+experiment+"/counts.mtx.gz"))
#     var = pd.read_csv(path+index+"/"+experiment+"/var.csv")
#     obs = pd.read_csv(path+index+"/"+experiment+"/obs.csv")
#     adata = sc.AnnData(X.T, var=var, obs=obs)
#     adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")
    adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")
        
    adata.var.index = adata.var["0"]
    mg = mygene.MyGeneInfo()
    query = mg.querymany(list(adata.var["0"]) , scopes='ensembl.gene', fields='symbol', species='human', returnall=True, verbose=False)
    
    df = pd.DataFrame(query['out'])
    df = df.set_index('query')
    df = df.drop_duplicates(keep='first')
    
    var=pd.merge(adata.var, df, how='left', left_index=True, right_index=True)
    var.loc[pd.isna(var['symbol']),'symbol'] = list(var.index[pd.isna(var['symbol'])])
    var=var[['symbol']]
    var = var.drop_duplicates(keep='first')
    adata.var['ensembl_id'] = adata.var.index
    adata.var_names=[var.loc[x]['symbol'] if x in var.index else x for x in adata.var_names]
    adata.var_names = [str(x) for x in adata.var.index.values]  # make strings, else not writeable
    adata.var.index = adata.var.index.rename('gene_symbol')
    adata.var = adata.var.drop(['Unnamed: 0','0'],axis=1)

    if experiment != "atscale":
        adata.obs = adata.obs.iloc[:,:15] # for lowMOI and highMOI 
    adata.obs['tissue_type']='cell_line'
    adata.obs['cell_line']='K562'
    adata.obs['cancer'] = True
    adata.obs['disease'] = 'chronic myelogenous leukemia'
    adata.obs['perturbation_type'] = 'CRISPR'
    adata.obs['celltype'] = 'lymphoblasts'
    adata.obs['organism'] = 'human'
    
    adata.obs = adata.obs.drop(['Unnamed: 0'],axis=1)
    adata.obs = adata.obs.rename({'cell': 'cell_barcode'}, axis=1)
    adata.obs['perturbation'] = [find_pert(x) for x in adata.obs['gene']]
    
    adata.obs.perturbation = ['control' if x.startswith("scrambled") else x for x in adata.obs.perturbation]
    
    adata.obs.cell_barcode = [x.split('-')[0] for x in adata.obs.cell_barcode]
    adata.obs.index = adata.obs.cell_barcode
    adata.obs.index = adata.obs.index.rename('cell_barcode')
    adata.obs = adata.obs.drop(['cell_barcode'],axis=1)
    
    if os.path.exists(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5"):
        !rm {path+index+"/"+experiment+"/"+index+"_"+experiment+".h5"}
    adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5")

... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation' as categorical


In [180]:
for experiment in experiments:
    !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+index+"_"+experiment+".h5"} {index+"/"+experiment+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/GasperiniShendure2019/lowMOI/GasperiniShendure2019_lowMOI.h5" to "/GasperiniShendure2019/lowMOI/GasperiniShendure2019_lowMOI.h5" by 24 chunks ........................ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/GasperiniShendure2019/highMOI/GasperiniShendure2019_highMOI.h5" to "/GasperiniShendure2019/highMOI/GasperiniShendure2019_highMOI.h5" by 28 chunks ............................ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/GasperiniShendure2019/atscale/GasperiniShendure2019_atscale.h5" to "/GasperiniShendure2019/atscale/GasperiniShendure2019_atscale.h5" by 115 chunks ................................................................................................................... DONE


# NormanWeissman2019

In [ ]:
# Done!

In [29]:
index = "NormanWeissman2019"

In [30]:
experiment = "filtered"

In [ ]:
X = csr_matrix(mmread(path+index+"/"+experiment+"/counts.mtx.gz"))
var = pd.read_csv(path+index+"/"+experiment+"/var.csv")
obs = pd.read_csv(path+index+"/"+experiment+"/obs.csv")
barcodes=pd.read_csv(path+index+"/"+experiment+"/GSE133344_filtered_barcodes.tsv.gz",header=None)
Xsub = X[:,[list(barcodes[0]).index(x) for x in obs.cell_barcode]]
adata = sc.AnnData(Xsub.T, var=var, obs=obs)
adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")

In [34]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")
adata.var.index = adata.var.gene_symbol
adata.var.index = adata.var.index.rename('gene_symbol')
adata.var = adata.var.drop(['Unnamed: 0','gene_symbol'],axis=1)

In [35]:
def identify_pert(guide):
    guide = guide.split("__")[0]
    perts = guide.split("_")
    nctrl = sum([x.startswith("NegCtrl") for x in perts])
    if nctrl == 2:
        pert = "control"
    elif nctrl == 1:
        pert = [x for x in perts if not x.startswith("NegCtrl")][0]
    else:
        pert = "_".join(perts)
    if pert == "no_reads_found":
        pert = "control"
    return pert

In [36]:
adata.obs['tissue_type']='cell_line'
adata.obs['cell_line']='K562'
adata.obs['cancer'] = True
adata.obs['disease'] = 'chronic myelogenous leukemia'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'lymphoblasts'
adata.obs['organism'] = 'human'

adata.obs['perturbation'] = [identify_pert(x) for x in adata.obs['guide_identity']]
adata.obs = adata.obs.rename({"guide_identity":"guide_id"},axis=1)
adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

# adata.obs = adata.obs.rename({'guide_identity': 'perturbation'}, axis=1)
# adata.obs.perturbation =[x.split('__')[0] for x in adata.obs.perturbation]
# adata.obs.perturbation = ['control' if x=='NegCtrl0_NegCtrl0' else x for x in adata.obs.perturbation]

In [37]:
adata.obs.cell_barcode = [x.split('-')[0] for x in adata.obs.cell_barcode]
adata.obs.index = adata.obs.cell_barcode
adata.obs.index = adata.obs.index.rename('cell_barcode')
adata.obs = adata.obs.drop(['Unnamed: 0','cell_barcode'],axis=1)

In [38]:
adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5")

... storing 'guide_id' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation' as categorical


In [39]:
experiment = "raw" #### file too big to process

In [199]:
X = csr_matrix(mmread(path+index+"/"+experiment+"/counts.mtx.gz"))
var = pd.read_csv(path+index+"/"+experiment+"/var.csv")
obs = pd.read_csv(path+index+"/"+experiment+"/obs.csv")
barcodes=pd.read_csv(path+index+"/"+experiment+"/GSE133344_raw_barcodes.tsv.gz",header=None)

In [200]:
X

<33694x5898240 sparse matrix of type '<class 'numpy.int64'>'
	with 442237190 stored elements in Compressed Sparse Row format>

In [208]:
Xsub = X[:,np.isin(barcodes[0], obs.cell_barcode)]

In [209]:
adata = sc.AnnData(Xsub.T, var=var, obs=obs)
adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")

Transforming to str index.
Transforming to str index.
... storing 'guide_identity' as categorical
... storing 'gene_symbol' as categorical


In [40]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_preprocessed.h5")

In [41]:
adata.var.index = adata.var.gene_symbol
adata.var.index = adata.var.index.rename('gene_symbol')
adata.var = adata.var.drop(['Unnamed: 0','gene_symbol'],axis=1)

In [42]:
adata.obs['tissue_type']='cell_line'
adata.obs['cell_line']='K562'
adata.obs['cancer'] = True
adata.obs['disease'] = 'chronic myelogenous leukemia'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'lymphoblasts'
adata.obs['organism'] = 'human'

adata.obs['perturbation'] = [identify_pert(x) for x in adata.obs['guide_identity']]
adata.obs = adata.obs.rename({"guide_identity":"guide_id"},axis=1)
adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

In [43]:
adata.obs.cell_barcode = [x.split('-')[0] for x in adata.obs.cell_barcode]
adata.obs.index = adata.obs.cell_barcode
adata.obs.index = adata.obs.index.rename('cell_barcode')
adata.obs = adata.obs.drop(['Unnamed: 0','cell_barcode'],axis=1)

In [44]:
adata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5")

... storing 'guide_id' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation' as categorical


In [40]:
for experiment in ["filtered","raw"]:
    !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+index+"_"+experiment+".h5"} {index+"/"+experiment+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/NormanWeissman2019/filtered/NormanWeissman2019_filtered.h5" to "/NormanWeissman2019/filtered/NormanWeissman2019_filtered.h5" by 56 chunks ........................................................ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/NormanWeissman2019/raw/NormanWeissman2019_raw.h5" to "/NormanWeissman2019/raw/NormanWeissman2019_raw.h5" by 57 chunks ......................................................... DONE


# TianKampmann2019

In [ ]:
# Done!

In [3]:
index = "TianKampmann2019"

In [46]:
sgrnanames = pd.read_csv(path+index+"/mmc3.csv")
sgrnanames.index = sgrnanames["protospacer sequence"]

In [47]:
experiments, idxs = ["day7neuron","iPSC"],["1","2"]

In [48]:
def find_sgrna(cellbarcode, sgrna):
    tmpsgrna = sgrna.loc[[cellbarcode],:]
    tmpsgrna = tmpsgrna.sort_values(['umi_count','read_count'], ascending=[False,False])
    sgRNA_barcodes = "__".join(tmpsgrna.guide)
    sgRNA_read_count = "-".join([str(x) for x in tmpsgrna.read_count])
    sgRNA_umi_count = "-".join([str(x) for x in tmpsgrna.umi_count])
    return [sgRNA_barcodes, sgRNA_read_count, sgRNA_umi_count]

In [49]:
def find_pert(guides):
    perts = list(set([x.split("_")[0] for x in guides]))
    if perts == ["control"]:
        return "control"
    elif "control" in perts:
        perts.remove("control")
        return "_".join(perts)
    else:
        return "_".join(perts)

In [52]:
sgrna = pd.read_csv(glob.glob(path+index+"/"+experiments[0]+"/"+experiments[0]+idxs[0]+"/*sgRNA_mapping.txt.gz")[0],sep="\t")

In [53]:
sgrna

,cell,barcode,read_count,umi_count
0,TCTCTAAAGACAGACC-1,unprocessed_TGTTGGTCCCGAGACTGATGTCCGGGTTTA,0,0
1,TCTCTAAAGACAGACC-1,GTACCACCCAAACGATAACG,0,0
2,TCTCTAAAGACAGACC-1,unprocessed_TGTTGGGAGCTCTCTGGCTAACTAGGGAAC,0,0
3,TCTCTAAAGACAGACC-1,GCTGGGAAGCTGGGTTCTCG,0,0
4,TCTCTAAAGACAGACC-1,GTGGCCTTACGAAGGAGCAG,3,1
...,...,...,...,...
1970722,GTTCTCGCAAAGTCAA-1,GACCGGATCGGTGACTGTGG,0,0
1970723,GTTCTCGCAAAGTCAA-1,unprocessed_TGTTGGCGTGGCAACTGGGTGAGGGGTTTA,0,0
1970724,CTCGAAAAGTATTGGA-1,GTACCACCCAAACGATAACG,1,1
1970725,CCGGGATAGGGTATCG-1,GACCCGAGACTGCTTCCCGG,1,1


In [54]:
for experiment in experiments:
    for idx in idxs:
#         X = csr_matrix(mmread(path+index+"/"+experiment+"/"+experiment+idx+"/counts.mtx.gz"))
#         var = pd.read_csv(path+index+"/"+experiment+"/"+experiment+idx+"/var.csv")
#         sgrna = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+experiment+idx+"/*sgRNA_mapping.txt.gz")[0],sep="\t")
        
#         sgrna.index = sgrna.cell
#         sgrna = sgrna.loc[sgrna['umi_count']>0,:]
#         sgrna = sgrna.loc[np.isin(sgrna.barcode,sgrnanames["protospacer sequence"]),:]
#         sgrna['guide'] = [sgrnanames.loc[x,'sgRNA_short name'] for x in tq.tqdm(sgrna['barcode'])]
#         sgrna['guide'] = ["control" if x.startswith("non-targeting") else x for x in sgrna['guide']]
        
#         barcodes = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+experiment+idx+"/*barcodes.tsv.gz")[0],sep="\t",
#                       header=None)
#         sharedcells = list(set(sgrna.index).intersection(list(barcodes[0])))
#         xsubindx = np.isin(list(barcodes[0]),sharedcells)
#         Xsub = X[:,xsubindx]
#         adata = sc.AnnData(Xsub.T)
#         adata.obs_names = barcodes.loc[xsubindx,0]
#         adata.var = var
        
#         obsinfo = [find_sgrna(x, sgrna) for x in tq.tqdm(adata.obs_names)]
#         adata.obs['guides'] = [x[0] for x in obsinfo]
#         adata.obs['sgRNA_read_count'] = [x[1] for x in obsinfo]
#         adata.obs['sgRNA_umi_count'] = [x[2] for x in obsinfo]

#         adata.write(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+"_preprocessed.h5")
        adata = sc.read_h5ad(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+"_preprocessed.h5")
        
        adata.var.index = adata.var.gene_symbol
        adata.var.index = adata.var.index.rename('gene_symbol')
        adata.var = adata.var.drop(['Unnamed: 0','gene_symbol'],axis=1)
        
        adata.obs['perturbation']=[find_pert(x.split("__")) for x in adata.obs["guides"]]
        if experiment == "day7neuron":
            adata.obs['tissue_type']='primary'
            adata.obs['celltype'] = 'iPSC-induced neuron'
        else: 
            adata.obs['tissue_type']='stem'
            adata.obs['celltype'] = 'iPSC'
        adata.obs['cancer'] = False
        adata.obs['disease'] = 'healthy'
        adata.obs['perturbation_type'] = 'CRISPR'
        adata.obs['organism'] = 'human'
        
        adata.obs = adata.obs.rename({"guides":"guide_id"},axis=1)
        adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

        adata.obs.index = [x.split("-")[0] for x in adata.obs.index]
        adata.obs.index = adata.obs.index.rename('cell_barcode')

        adata.write(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5")


Transforming to str index.
... storing 'perturbation' as categorical
... storing 'guide_id' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'organism' as categorical
Transforming to str index.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'organism' as categorical
Transforming to str index.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'organism' as categorical
Transforming to str index.
... storing 'guide_id' as categorical
... storing

In [ ]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+"_preprocessed.h5")


In [27]:
## combine data from different batch
for experiment in ["day7neuron","iPSC"]:
    for idx in ['1','2']:
        adata = sc.read_h5ad(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5")
        #adata.var_names_make_unique()
        if idx=='1':
            adata.obs['batch']=1
            fulladata = adata
        else:
            adata.obs['batch']=2
            fulladata = sc.AnnData(vstack((fulladata.X,adata.X),"csc"),
                                   obs=pd.concat([fulladata.obs,adata.obs]), var=fulladata.var)
            fulladata.obs_names_make_unique()
    fulladata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/cs308/cellbox_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'sgRNA_read_count' as categorical
... storing 'sgRNA_umi_count' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them 

In [187]:
for experiment in experiments:
    for idx in idxs:
        !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5"} {index+"/"+experiment+"/"+experiment+idx+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2019/day7neuron/day7neuron1/TianKampmann2019_day7neuron1.h5" to "/TianKampmann2019/day7neuron/day7neuron1/TianKampmann2019_day7neuron1.h5" by 10 chunks .......... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2019/day7neuron/day7neuron2/TianKampmann2019_day7neuron2.h5" to "/TianKampmann2019/day7neuron/day7neuron2/TianKampmann2019_day7neuron2.h5" by 9 chunks ......... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2019/iPSC/iPSC1/TianKampmann2019_iPSC1.h5" to "/TianKampmann2019/iPSC/iPSC1/TianKampmann2019_iPSC1.h5" by 12 chunks ............ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2019/iPSC/iPSC2/TianKampmann2019_iPSC2.h5" to "/TianKampmann2019/iPSC/iPSC2/TianKampmann2019_iPSC2.h5" by 12 chunks ............ DONE


# TianKampmann2021

In [ ]:
# Done!

In [28]:
index = "TianKampmann2021"

In [29]:
experiments = [["CRISPRa","1"],["CRISPRa","2"],["CRISPRi","1"],["CRISPRi","2"],
               ["CRISPRi","3"],["CRISPRi","4"]]

In [57]:
for experiment,idx in experiments:
    adata=sc.read_h5ad(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+"_preprocessed.h5")
    adata.var.index = adata.var["name"]
    adata.var = adata.var.rename({'id': 'ensembl_id'}, axis=1)
    adata.var.index = adata.var.index.rename('gene_symbol')
    adata.var = adata.var.drop(['name'],axis=1)
    
    maps=pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+experiment+idx+"/mapped*txt")[0],sep="\t",header=None)
    maps.index=maps[0]
    adata.obs.index = adata.obs.barcode
    mapped = adata.obs.merge(maps, left_index=True, right_index=True, how="inner")
    mapped_cells = mapped.index
    
    adata = sc.AnnData(adata.X[adata.obs.index.isin(mapped_cells),],var=adata.var,obs=mapped)
    #adata.var_names_make_unique
    adata.obs = adata.obs.rename({1: 'guide'}, axis=1)
    adata.obs["perturbation"] = [x.split("_")[0] for x in adata.obs.guide]
    adata.obs["perturbation"] = ["control" if x.startswith("non-target") else x for x in adata.obs["perturbation"]]
    
    adata.obs['tissue_type']='primary'
    adata.obs['celltype'] = 'iPSC-induced neuron'
    adata.obs['cancer'] = False
    adata.obs['disease'] = 'healthy'
    adata.obs['perturbation_type'] = 'CRISPR'
    adata.obs['organism'] = 'human'
    
    adata.obs = adata.obs.rename({"guide":"guide_id"},axis=1)
    adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]
    
    adata.obs = adata.obs.drop(["barcode",0], axis=1)
    adata.obs.index = [x.split("-")[0] for x in adata.obs.index]
    adata.obs.index = adata.obs.index.rename('cell_barcode')
    
    adata.write(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'organism' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'organism' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'guide_id' as categorical
... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'celltype' as categorical
... storing 'disease' as categ

In [30]:
## combine data from different batch
for experiment,idx in experiments:
    adata=sc.read_h5ad(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5")
    adata.obs['batch']=idx
        #adata.var_names_make_unique()
    if idx=='1':
        fulladata = adata
    else:
        fulladata = sc.AnnData(vstack((fulladata.X,adata.X),"csc"),
                               obs=pd.concat([fulladata.obs,adata.obs]), var=fulladata.var)
        fulladata.obs_names_make_unique()
    fulladata.write(path+index+"/"+experiment+"/"+index+"_"+experiment+".h5")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'batch' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'batch' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'batch' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'guide_id' as categorical
... storing 'batch' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_nam

In [190]:
for experiment,idx in experiments:
    !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+".h5"} {index+"/"+experiment+"/"+experiment+idx+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2021/CRISPRa/CRISPRa1/TianKampmann2021_CRISPRa1.h5" to "/TianKampmann2021/CRISPRa/CRISPRa1/TianKampmann2021_CRISPRa1.h5" by 6 chunks ...... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2021/CRISPRa/CRISPRa2/TianKampmann2021_CRISPRa2.h5" to "/TianKampmann2021/CRISPRa/CRISPRa2/TianKampmann2021_CRISPRa2.h5" by 6 chunks ...... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2021/CRISPRi/CRISPRi1/TianKampmann2021_CRISPRi1.h5" to "/TianKampmann2021/CRISPRi/CRISPRi1/TianKampmann2021_CRISPRi1.h5" by 6 chunks ...... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2021/CRISPRi/CRISPRi2/TianKampmann2021_CRISPRi2.h5" to "/TianKampmann2021/CRISPRi/CRISPRi2/TianKampmann2021_CRISPRi2.h5" by 6 chunks ...... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/TianKampmann2021/CRISPRi/CRISPRi3/TianKampmann2021_CRISPRi3.h5

# McFarlandTshemiak2020

In [ ]:
# in its own folder Done!

In [191]:
index = "McFarlandTshemiak2020"

In [194]:
for folder in os.listdir(path+index+"/"):
    if "expt" in folder:
        !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+folder+"/"+folder+".h5"} {index+"/"+folder+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/McFarlandTshemiak2020/Navitoclax_24hr_expt3/Navitoclax_24hr_expt3.h5" to "/McFarlandTshemiak2020/Navitoclax_24hr_expt3/Navitoclax_24hr_expt3.h5" by 8 chunks ........ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/McFarlandTshemiak2020/BRD3379_24hr_expt3/BRD3379_24hr_expt3.h5" to "/McFarlandTshemiak2020/BRD3379_24hr_expt3/BRD3379_24hr_expt3.h5" by 8 chunks ........ DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/McFarlandTshemiak2020/AZD5591_expt10/AZD5591_expt10.h5" to "/McFarlandTshemiak2020/AZD5591_expt10/AZD5591_expt10.h5" by 4 chunks .... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/McFarlandTshemiak2020/Taselisib_expt10/Taselisib_expt10.h5" to "/McFarlandTshemiak2020/Taselisib_expt10/Taselisib_expt10.h5"... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/McFarlandTshemiak2020/Everolimus_expt10/Everolimus_expt10.h5" to "/McFarlandTshemiak2020

# PapalexiSatija2021

In [ ]:
# Done!

In [31]:
index = "PapalexiSatija2021"

## eccite_arrayed

In [60]:
experiment = "eccite_arrayed"

In [5]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_RNA_preprocessed.h5")
adata.var.index = adata.var.gene

In [6]:
hto = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO_counts*")[0],
                  sep="\t",index_col=0)
hto_bc1 = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO1_Barcodes*")[0],
                     sep=",",index_col=0,header=None)
hto_bc2 = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO2_Barcodes*")[0],
                     sep=",",index_col=0,header=None)
gdo = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*GO_CITE*_counts*")[0],
                  sep="\t",index_col=0)
gdo_bc = pd.read_csv(glob.glob(path+index+"/eccite/"+"GSM*GDO_Barcodes*")[0],
                     sep=",",index_col=0,header=None)

In [8]:
mg = mygene.MyGeneInfo()
query = mg.querymany(list(adata.var["gene"]) , scopes='symbol', fields='ensembl.gene', species='human', returnall=True, verbose=False)
gene_symbols = [x['query'] for x in query['out']]
ensembl_id = [x['ensembl'] if 'ensembl' in x.keys() else np.nan for x in query['out']]
ensembl_id = [x['gene'] if type(x)==dict else x for x in ensembl_id]
ensembl_id = [x[0]['gene'] if type(x)==list else x for x in ensembl_id]

df = pd.DataFrame({"gene":gene_symbols,"ensembl_id":ensembl_id})
df.index = gene_symbols
df = df.drop_duplicates(keep='first')

In [12]:
for dupidx in np.where(df.index.duplicated())[0]:
    ensemblids = df.iloc[dupidx-1:dupidx+1]["ensembl_id"]
    if sum(~pd.isna(ensemblids)) == 1:
        ensemblid = ensemblids[~pd.isna(ensemblids)]
        df.iloc[dupidx-1:dupidx+1]["ensembl_id"] = ensemblid
df = df.drop_duplicates(subset="gene", keep='first')

In [70]:
adata.var["ensembl_id"] = df.loc[adata.var.index,"ensembl_id"]
adata.var.loc[pd.isna(adata.var['ensembl_id']),'ensembl_id'] = adata.var.gene[pd.isna(adata.var['ensembl_id'])]
adata.var.index = adata.var.index.rename('gene_symbol')
adata.var = adata.var.drop(['gene'],axis=1)

In [7]:
def define_target(guide):
    if guide.startswith("NT") or guide=="CTRL":
        target = "CTRL"
    else:
        idx = guide.find("g")
        if idx == -1:
            target = guide
        else:
            target = guide[:idx]
    if target == "IFNG":
        target = "IFNGR2"
    return target

In [8]:
htoguides = [define_target(x.split("-")[1]) for x in hto.idxmax()]
gdoguides = [define_target(x) for x in gdo.idxmax()]

In [9]:
idxkeep1 = np.where(gdo.max()>=5)[0] # negative cells
idxkeep2 = np.where(np.array(htoguides) == np.array(gdoguides))[0] 
# consistent perturbation between HTO and GDO
idxkeep = np.intersect1d(idxkeep1,idxkeep2)

In [11]:
hto_bc = pd.concat([hto_bc1,hto_bc2])
hto_bc["barcode"] = hto_bc.index
hto_bc.index = ["-".join(x.split("_")) for x in hto_bc[1]]

In [12]:
adata.obs.index = [x.split("_")[1] for x in adata.obs.cell_barcode]
adata.obs["perturbation"] = gdoguides
adata.obs["hto"] = list(hto.idxmax())
adata.obs["gdo"] = list(gdo.idxmax())
adata.obs["hto"] = ["HTO-NT7" if x.startswith("HTO-NT7") else x for x in adata.obs["hto"]]
adata.obs["hto_barcode"] = [hto_bc.loc[x,"barcode"] for x in adata.obs["hto"]]

In [126]:
# hto_bc.index = hto_bc[1]

In [13]:
gdo_bc["barcode"] = gdo_bc.index
gdo_bc.index = gdo_bc[1]

In [14]:
adata.obs["perturbation"] = ["control" if x=="CTRL" else x for x in adata.obs["perturbation"]]
adata.obs["gdo_barcode"] = [gdo_bc.loc[x,"barcode"] for x in adata.obs["gdo"]]
adata.obs['tissue_type']='cell_line'
adata.obs['cell_line']='THP-1'
adata.obs['cancer'] = True
adata.obs['disease'] = 'acute monocytic leukemia'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'monocytes'
adata.obs['organism'] = 'human'

In [15]:
adata.obs = adata.obs.drop(["cell_barcode"], axis=1)
adata.obs.index = adata.obs.index.rename('cell_barcode')

In [16]:
obs = adata.obs

In [161]:
adata = sc.AnnData(adata.X[idxkeep,:],obs=adata.obs.iloc[idxkeep,:],var=adata.var)
adata.obs_names_make_unique()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [61]:
# adata = sc.read_h5ad(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA.h5")

# adata.obs = adata.obs.rename({"gdo":"guide_id"},axis=1)
# adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA.h5")

... storing 'guide_id' as categorical


In [19]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_protein_preprocessed.h5")
adata.var.index = adata.var.protein

In [17]:
adt_bc = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*ADT_Barcodes*")[0],
                     sep=",",index_col=0,header=None)

In [30]:
adata.var["adt_barcode"] =  [adt_bc.index[adt_bc[1] == x][0] for x in adata.var.index]
adata.var = adata.var.drop(["protein"], axis=1)

In [34]:
adata.obs = obs
adata = sc.AnnData(adata.X[idxkeep,:],obs=adata.obs.iloc[idxkeep,:],var=adata.var)
adata.obs_names_make_unique()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [62]:
# adata = sc.read_h5ad(index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5")

# adata.obs = adata.obs.rename({"gdo":"guide_id"},axis=1)
# adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5")

... storing 'guide_id' as categorical


In [63]:
adata = sc.read_h5ad(index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5")

## eccite

In [65]:
experiment = "eccite"

In [157]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_RNA_preprocessed.h5")
adata.var.index = adata.var.gene

In [158]:
adt_bc = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*ADT_Barcodes*")[0],
                     sep=",",index_col=0,header=None)
hto = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO_counts*")[0],
                  sep="\t",index_col=0)
hto_bc1 = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO_1-4_Barcodes*")[0],
                     sep=",",index_col=0,header=None)
hto_bc2 = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*HTO_5-8_Barcodes*")[0],
                     sep=",",index_col=0,header=None)
gdo = pd.read_csv(glob.glob(path+index+"/"+experiment+"/"+"GSM*GDO_counts*")[0],
                  sep="\t",index_col=0)
gdo_bc = pd.read_csv(glob.glob(path+index+"/eccite/"+"GSM*GDO_Barcodes*")[0],
                     sep=",",index_col=0,header=None)

In [159]:
mg = mygene.MyGeneInfo()
query = mg.querymany(list(adata.var["gene"]) , scopes='symbol', fields='ensembl.gene', species='human', returnall=True, verbose=False)
gene_symbols = [x['query'] for x in query['out']]
ensembl_id = [x['ensembl'] if 'ensembl' in x.keys() else np.nan for x in query['out']]
ensembl_id = [x['gene'] if type(x)==dict else x for x in ensembl_id]
ensembl_id = [x[0]['gene'] if type(x)==list else x for x in ensembl_id]

df = pd.DataFrame({"gene":gene_symbols,"ensembl_id":ensembl_id})
df.index = gene_symbols
df = df.drop_duplicates(keep='first')

for dupidx in np.where(df.index.duplicated())[0]:
    ensemblids = df.iloc[dupidx-1:dupidx+1]["ensembl_id"]
    if sum(~pd.isna(ensemblids)) == 1:
        ensemblid = ensemblids[~pd.isna(ensemblids)]
        df.iloc[dupidx-1:dupidx+1]["ensembl_id"] = ensemblid
df = df.drop_duplicates(subset="gene", keep='first')

adata.var["ensembl_id"] = df.loc[adata.var.index,"ensembl_id"]
adata.var.loc[pd.isna(adata.var['ensembl_id']),'ensembl_id'] = adata.var.gene[pd.isna(adata.var['ensembl_id'])]
adata.var.index = adata.var.index.rename('gene_symbol')
adata.var = adata.var.drop(['gene'],axis=1)

/home/cs308/cellbox_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [160]:
htoguides = list(hto.idxmax())
gdoguides = ["control" if x.startswith("NT") else x for x in gdo.idxmax()]

In [161]:
hto_bc = pd.concat([hto_bc1,hto_bc2])
hto_bc["barcode"] = hto_bc.index
hto_bc.index = ["-".join(x.split("_")) for x in hto_bc[1]]

In [162]:
gdo_bc["barcode"] = gdo_bc.index
gdo_bc.index = gdo_bc[1]

In [163]:
adata.obs["hto"] = htoguides
adata.obs["gdo"] = list(gdo.idxmax())
adata.obs["hto_barcode"] = [hto_bc.loc[x,"barcode"] for x in adata.obs["hto"]]
adata.obs["gdo_barcode"] = [gdo_bc.loc[x,"barcode"] for x in adata.obs["gdo"]]

In [164]:
adata.obs["perturbation"] = gdoguides
adata.obs['tissue_type']='cell_line'
adata.obs['cell_line']='THP-1'
adata.obs['cancer'] = True
adata.obs['disease'] = 'acute monocytic leukemia'
adata.obs['perturbation_type'] = 'CRISPR'
adata.obs['celltype'] = 'monocytes'
adata.obs['organism'] = 'human'

In [165]:
adata.obs.index = [x.split("_")[1] for x in adata.obs["cell_barcode"]]
adata.obs = adata.obs.drop(["cell_barcode"], axis=1)
adata.obs.index = adata.obs.index.rename('cell_barcode')

In [166]:
obs = adata.obs

In [66]:
# adata = sc.read_h5ad(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA.h5")

# adata.obs = adata.obs.rename({"gdo":"guide_id"},axis=1)
# adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA.h5")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'guide_id' as categorical


In [169]:
adata = sc.read_h5ad(path+index+"/"+experiment+"/"+index+"_"+experiment+"_protein_preprocessed.h5")
adata.var.index = adata.var.protein

In [170]:
adata.var["adt_barcode"] =  [adt_bc.index[adt_bc[1] == x][0] for x in adata.var.index]
adata.var = adata.var.drop(["protein"], axis=1)

In [171]:
adata.obs = obs

In [67]:
# adata = sc.read_h5ad(index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5")

# adata.obs = adata.obs.rename({"gdo":"guide_id"},axis=1)
# adata.obs["guide_id"] = [";".join(x.split("__")) for x in adata.obs["guide_id"]]

adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'guide_id' as categorical


In [196]:
for experiment in ["eccite_arrayed", "eccite"]:
    !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+index+"_"+experiment+"_protein.h5"} {index+"/"+experiment+"/"}
    !/home/cs308/dropbox_uploader.sh upload {path+index+"/"+experiment+"/"+index+"_"+experiment+"_RNA.h5"} {index+"/"+experiment+"/"}

 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/PapalexiSatija2021/eccite_arrayed/PapalexiSatija2021_eccite_arrayed_protein.h5" to "/PapalexiSatija2021/eccite_arrayed/PapalexiSatija2021_eccite_arrayed_protein.h5"... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/PapalexiSatija2021/eccite_arrayed/PapalexiSatija2021_eccite_arrayed_RNA.h5" to "/PapalexiSatija2021/eccite_arrayed/PapalexiSatija2021_eccite_arrayed_RNA.h5" by 4 chunks .... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/PapalexiSatija2021/eccite/PapalexiSatija2021_eccite_protein.h5" to "/PapalexiSatija2021/eccite/PapalexiSatija2021_eccite_protein.h5"... DONE
 > Uploading "/n/data1/hms/cellbio/sander/judy/resource_paper/PapalexiSatija2021/eccite/PapalexiSatija2021_eccite_RNA.h5" to "/PapalexiSatija2021/eccite/PapalexiSatija2021_eccite_RNA.h5" by 11 chunks ........... DONE
